In [1]:
import pyodbc, time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
# BD Methods:

def createConexion(DB_HOST, DB_NAME, DB_USER, DB_PASSWORD):
    connectionString = f'''DRIVER={{ODBC Driver 18 for SQL Server}};
    SERVER={DB_HOST};DATABASE={DB_NAME};
    UID={DB_USER};PWD={DB_PASSWORD};
    Encrypt=Yes;TrustServerCertificate=Yes'''

    return pyodbc.connect(connectionString)

def dbSelect(conexion, table):
    SQL_QUERY = f"SELECT * FROM {table}"

    #Creamos un cursos para logo acceder aos resultados
    cursor = conexion.cursor()

    #Finalmente executamos a consulta
    cursor.execute(SQL_QUERY)

    tuplas = cursor.fetchall()
    for tupla in tuplas:
        campos = ""
        for campo in tupla:
            campos += f"{campo}\t"
        print(f"{campos}")

    cursor.close()
    conexion.close()

def dbInsertMasivoEmpregado(conexion):
    import traceback

    SQL_PREPARED_STATEMENT = """
        INSERT INTO empregado(nome, apelidos, dni, data_creacion)
        VALUES (?,?,?, CURRENT_TIMESTAMP);
    """

    conexion = pyodbc.connect(conexion)
    cursor = conexion.cursor()

    try:
        for sufijo in range(0, 10):

            cursor.execute(
                SQL_PREPARED_STATEMENT,
                f'nome_{sufijo}',
                f'apelidos_{sufijo}',
                f'{sufijo}2345678A')
            conexion.commit()

    except Exception as e:
        # Print the error and traceback
        print(e)
        traceback.print_exc()
        # Rollback in case of error
        conexion.rollback()
    finally:
        # Close the cursor and connection
        cursor.close()
        conexion.close()

# Scraping Methods:

def clickRowByName(container, name, i = 0):
    for row in container.find_elements(By.TAG_NAME, "tr"):
        element = row.find_element(By.CLASS_NAME, "tafelTreecontent")
        if (element.text == name):
            if (i <= 0):
                    row.find_element(By.CLASS_NAME, "tafelTreeopenable").click()
                    return
            else:
                i-=1
        
def clickLabelByName(container, name, i = 0):
    for row in container.find_elements(By.TAG_NAME, "tr"):
        element = row.find_element(By.CLASS_NAME, "tafelTreecontent")
        if (element.text == name):
            if (i <= 0):
                    element.click()
                    return
            else:
                i-=1

def calcularLinks(driver, anhadir):
    result = list()
    for link in anhadir:
        driver.get(link)
        nueva_fila = dict()

        nueva_fila["id"] =  driver.find_element(By.ID, "fila0_columna0").find_elements(By.TAG_NAME, "span")[1].text

        datos = driver.find_elements(By.CLASS_NAME, "capaAtributos")
        for dato in datos:
            for row in dato.find_elements(By.TAG_NAME, "ul"):
                ils = row.find_elements(By.TAG_NAME, "li")

                clave = ils[0].find_element(By.TAG_NAME, "span").text.replace("\"", "'")
                valor = ils[1].find_element(By.TAG_NAME, "span").text.replace("\"", "'")

                nueva_fila[clave] = valor

        nueva_fila["id"] =  driver.find_element(By.ID, "fila0_columna0").find_elements(By.TAG_NAME, "span")[1].text

        result.append(nueva_fila)
        
    return result

In [3]:
# DB Definitions:
DB_HOST = '127.0.0.1,41433'
DB_NAME = 'master'
DB_USER = 'sa'
DB_PASSWORD = 'Abc12300'

# Scraping Definitions:
URL="https://contrataciondelestado.es/wps/portal/plataforma"

try:
    def cambiarPantalla():
        time.sleep(1)
        driver.get(driver.execute_script("return window.location.href;"))
        time.sleep(1)

    driver = webdriver.Firefox()
    driver.get(URL)
        
    time.sleep(1)

    element = driver.find_elements(By.CLASS_NAME, "card-item")[0].click()

    # Licitaciones
    licitaciones = driver.find_element(By.XPATH, "/html/body/div/div/div[2]/div[2]/div[1]/div/div/div[1]/div/form/div[1]/div[1]/a/p[2]/span")
    licitaciones.click()
    time.sleep(0.5)

    # Búsqueda avanzada
    cosas = driver.find_elements(By.CLASS_NAME, "rutaGuiada")
    busquedaAvanzada = None
    for item in cosas[:2]:
        if item.text == "Búsqueda avanzada":
            busquedaAvanzada = item
    busquedaAvanzada.click()

    time.sleep(0.2)

    # Seleccionar
    seleccionar = driver.find_element(By.XPATH, "//span[contains(text(), 'Seleccionar')]")
    seleccionar.click()

    time.sleep(1)

    div = driver.find_elements(By.CLASS_NAME, "tafelTree_row")[2]
    img = div.find_element(By.CLASS_NAME, "tafelTreeopenable")
    img.click()

    time.sleep(0.2)
    clickRowByName(div, "Galicia")
    time.sleep(0.2)
    clickRowByName(div, "A Coruña")
    time.sleep(0.2)
    clickRowByName(div, "Ayuntamientos")
    time.sleep(0.2)
    clickLabelByName(div, "A Coruña", 1)
    time.sleep(0.2)

    cuadro = driver.find_element(By.XPATH, "//div//span//select")
    opciones = cuadro.find_elements(By.TAG_NAME, "option")

    for opcion in opciones:
        if(opcion.get_attribute("value") == "93655277"):
            opcion.click()

    driver.find_element(By.XPATH, "//input[@value='Añadir']").click()
    time.sleep(1)
    driver.find_element(By.XPATH, "//input[@value='Buscar']").click()

    time.sleep(5)

    anhadir = list()
    while(True):
        table = driver.find_element(By.ID, "myTablaBusquedaCustom")
        rows = table.find_elements(By.TAG_NAME, "tr")

        for row in rows[1:]:
            try:
                expediente = row.find_element(By.CLASS_NAME, "tdExpediente")
                divs = expediente.find_elements(By.TAG_NAME, "div")
                anhadir.append(divs[0].find_elements(By.TAG_NAME, "a")[1].get_attribute("href"))

            except(Exception) as e:
                # Existen algunas "tr" que tienen formato extraño 
                # así que seguimos la táctica "Let It Crash"
                pass

        element = driver.find_element(By.XPATH, "//input[contains(@class, 'botonEnlace') and @title='Siguiente']")
        if (element != None):     
            element.click()
            time.sleep(0.5)

except(Exception) as error:
    print("Error: ", error)

result = calcularLinks(driver, anhadir)

driver.quit()

print("Terminé!")

# conexion = createConexion(DB_HOST, DB_NAME, DB_USER, DB_PASSWORD)

# dbSelect(conexion, "empregado")

The geckodriver version (0.35.0) detected in PATH at C:\Users\gabriel.antonio.edre\mis-binarios\geckodriver.exe might not be compatible with the detected firefox version (136.0.2.571); currently, geckodriver 0.36.0 is recommended for firefox 136.*, so it is advised to delete the driver in PATH and retry


Error:  Message: Unable to locate element: //input[contains(@class, 'botonEnlace') and @title='Siguiente']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:197:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:527:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Terminé!


In [ ]:
import json

# Escribir el archivo
with open("result.json", "w", encoding="utf-8") as archivo:
    json.dump(result, archivo, indent=4, ensure_ascii=False)